# Getting Started with Datacube Stats
| Author(s):  | [Arapaut Sivaprasad](mailto:Sivaprasad.Arapaut@ga.gov.au)|
|----------|----------------|
| Created: | May 17, 2018 |
| Last edited: | May 23, 2018 |
| Acknowledgements: | Imam Alam|

## About this document

**Background**

Data Cube Statistics is a an application to calculate large scale temporal statistics on data stored using an Open Data Cube (ODC) installation. It provides a command line application which uses a YAML configuration file to specify the data range and statistics to calculate.

**What does this document do?**

This document is a startup guide to using the 'document-stats' program to analyse continental scale statistics over a 30+ year time range. It aims to get you started using the program, with basic examples and statistical analyses. Please see **Advanced Users Guide for Datacube-stats** for more complex analyses and options.

The program is intended to be run from commandline, or as a queue job, on Raijin. It takes several minutes to several hours to complete the job. However, in order to understand the usage and to see the output in a visual manner, this document will show an interactive session to analyse just one scene or "tile", which takes only a few seconds. Below it will be given the detailed instructions to run the program on Raijin interactively and through queue jobs.

A basic understanding of the PSB job scheduler on Raijin is advantageous, but not essential, to follow the instructions in this document. You must have, as expected to be, an account on Raijin with memebership in at least one project. Please verify with your supervisor about which project to use and how much resources are allocated to your research. The costs will vary depending on the type of PBS queue used as well as the complexity and size of the job.

**How to use this document**

There are five sections that describe the software and its usage. You can skip any section by clicking its heading line.

1. Backgound information about the software and its inputs and outputs.
2. Run interactively in a Unix shell and and monitor the progress.
3. Submit to a PBS job queue and monitor the progress.
4. Run interactively through Jupyter notebook.
5. Visualise the outputs.

## SECTION 1: Background on 'Datacube-stats' Application


### Data Cube Statistics Tools
Data Cube Statistics is an application to calculate large scale temporal statistics on data stored using an Open Data Cube (ODC) installation. It provides a command line application which uses a YAML configuration file to specify the data range and statistics to calculate.

#### Main Features

- Calculate continental scale statistics over a 30+ year time range.
- Simple yet powerful Configuration format.
- Scripts supplied for running in a HPC PBS based environment.
- Flexible tiling and chunking for efficient execution of 100+Tb jobs.
- Track and store full provenance record of operations.
- Round-trip workflow from ODC back into ODC.
- Supports saving to NetCDF, GeoTIFF or other GDAL supported format.
- Optional per-pixel metadata tracking.
- Out of the box support for most common statistics - see Available statistics.
- Able to create user defined Custom statistics.
- Able to handle any CRS and resolution combination (through the power of the ODC).

#### Usage
This is a commandline-driven program and all configurations are specified in a YAML file. The simplest way to execute it is as below.

> $ datacube-stats example-configuration.yaml




#### Available Statistics
The following statistical methods are implemented.

- simple
- percentile
- percentile_no_prov
- medoid
- medoid_no_prov
- medoid_simple
- simple_normalised_difference
- none
- wofs_summary
- tcwbg_summary
- masked_multi_count
- external
- wofs-summary
- geomedian

## SECTION 2: Interactive execution on Raijin or VDI

This program is meant to be run as a batch job, since it may take several minutes to hours to complete. However, when the date range is small, you may want to execute it interactively and see its progress. A commandline execution is possible in this situation.

#### Run in serial mode
In serial mode, the execution happens on just one core (CPU) and uses as much RAM as required. There is no need, or facility, to specify the number of cores and RAM in serial mode.

To run in serial mode, the following command is issued at the shell prompt.

> ***datacube-stats config_file***, where 'config_file' is a properly formatted YAML input file (see below)

e.g. *datacube-stats working_example_1.yaml*


##### working_example_1.yaml
A simple example to load and analyse all scenes for a date range is given in the example YAML file below. It can be run by the following commandline switches:

- **datacube-stats working_example_1.yaml:** To create an output file for each of the tiles. The date range below results in **89** tiles.

- **datacube-stats --tile-index 15 -40 working_example_1.yaml:** Create just one output file for the specified tile.

**Running interactively on the login node is inefficient and error-prone for the following reasons.** Avoid it if possible.
- Competes with other processes on the node. 
    - It means that other users on the system could be affected by your process.
    - Your process may get terminated by the system for using excessive resources.
    - On raijin, a process that runs more than 30 minutes on the login node is automatically terminated.
- Your process may not get enough RAM to run and will exit.
    - This results in frequent "Memory Error"
- It will take extremely long time to finish, if at all runs without errors, as the resources are small.
- Use it only if you are analysing a single scene as below.
    - **datacube-stats --tile-index 15 -40 working_example_1.yaml:** Create just one output file for the specified tile.

**Run interactively in a PBS queue**

If you require to run interactively, for any reason such as development or analysis, there is a facility to create an interactive session through a queue job. It will allocate one or more nodes for your exclusive use, and you can execute the program interactively as you would on any login session,l but without competing for resources.

To start an interactive queue sesssion, issue one of the following commands at the shell prompt.

- **qsub -I -qexpress -lwalltime=01:00:00,ncpus=16,mem=32GB,jobfs=300GB,wd** : To use 1 node and 16 CPUs and 32 GB RAM
- **qsub -I -qexpress -lwalltime=01:00:00,ncpus=32,mem=64GB,jobfs=300GB,wd** : To use 2 nodes with 32 CPUs and 64 GB RAM
- etc.

You must remember to request a longer session time, **-lwalltime=10:00:00** if the job is expected to run for longer than 1 hour. Otherwise, the job may get killed when the requested session time exceeds. If the job finishes in less time, but you do not kill the queue job, it will keep racking up the wall time and get charged for the full 10 hours.



#### Run in parallel mode

Running the program in parallel mode will speed up the execution almost proportionately as the number of cores (CPUs) used. To do it, all that is to be done is add **--parallel N**, where N is the number of cores to be used. When you have started an interactive queue session as described in the previous section, you can specify N as a number equal to or less than the CPUs requested.

e.g. **datacube-stats --parallel 16 working_example_1.yaml:** To parallelise the execution on 16 cores.

Arbitrarily increasing the cores in your request will not result in linearly increased performance. After certain number of cores the performance actually goes down. This is due to the fact that communication and coordination between cores consume resources and time. It is a variable that you can only arrive at by trial and error. The speedup is based on **[Amdahl's Law](https://en.wikipedia.org/wiki/Amdahl%27s_law)**, and is depended on the level and type of parallelisation in the code. With the example YAML file given above, it has been observed that the peak performance is at **16** cores. If requesting more than 16 cores, in the **--parallel N** option, the execution time actually goes up considerably. It is, presumably, due to the fact that a node has only 16 cores and, therefore, requesting more than 16 results in communication overhead between nodes (*please comment!*).  

## Errors and warnings

1. **Memory Exceeded:** This is commonly seen when the date range is anything more than a few days. Running on a PBS queue with 16 cores and 32 GB could only safely run 2 days' date range. When the date range was increased to 1 month, the following errors started to happen.

**Running the job on an interactive queue invoked by the '*qsub -I -qexpress -lwalltime=01:00:00,ncpus=16,mem=128GB,jobfs=300GB,wd*' gave the errors below, irrespective of the 'ncpus' and 'mem' used.**

- PBS: job killed: mem job total 134217728 kb exceeded limit 134217728 kb

> Job 7122684.r-man2 has exceeded memory allocation on node r3567

> HDF5-DIAG: Error detected in HDF5 (1.10.1) thread 140385198302976:

> 2018-05-23 09:14:06,258 15574 datacube.storage.storage ERROR Error opening source dataset: NetCDF:/g/data/rs0/datacube/002/LS8_OLI_NBAR/17_-32/LS8_OLI_NBAR_3577_17_-32_2013_v1496416918.nc:swir1
2018-05-23 09:14:08,392 15574 datacube.utils WARNING Ignoring Exception: Read or write failed
2018-05-23 09:14:13,086 15538 datacube_stats.main ERROR A child process terminated abruptly, the process pool is not usable anymore

> #008: H5FDsec2.c line 715 in H5FD_sec2_read(): file read failed: time = Wed May 23 09:14:01 2018
, filename = '/g/data/rs0/datacube/002/LS8_OLI_NBAR/17_-32/LS8_OLI_NBAR_3577_17_-32_2013_v1496416918.nc', file descriptor = 38, errno = 12, error message = 'Cannot allocate memory', buf = 0x92f506f, total read size = 12693, bytes this sub-read = 12693, bytes actually read = 18446744073709551615, offset = 2349871104

**Running a batch job with the same YAML file did not cause any error.** 

- Memory Requested:   64.0GB;   Memory Used: 27.99GB; NCPUs Used: 16
- Number of tiles: 1,415

It means that somehow the interactive job, even though using dedicated resources, is exceeding the RAM uasge. 

This disparity between interactive and batch jobs does not make any obvious sense, and requires a deep investigation into the code. 

Unless extreme care is taken in managing the memory, it is possible to fill up the RAM. I have encountered it when writing a pattern matching algorithm where several billions of words were to be analysed. The solutions Python and Perl provided for that problem were very elgant. We might look into it to see if it is applicable in the current situation.

Even though there is a RAM issue as above, there may not be any practical impact of it. The sample YAML was trying to use ALL tiles taken during a specified range within the  Spatial Reference EPSG:3577 (i.e. whole of Australia). I do not think it will ever be an intention to take all of them (*please comment!*). Instead, if we specify the particular tiles we are interested in (e.g. tiles 15,-40 plus 15,-41 to cover all of Canberra) there may not be any issue. This option, however, is not specifiable in the YAML but is a commandline option. Getting one tile in a 2-day window took less than a few seconds. Perhaps for a longer date range it will take longer, but still not hours, and no RAM exceeded error may occur. We can list as many tiles as we need in a plain text file and use it alongwith the '--tile-index-file' param.

**Error Mitigation Strategies:**

**1.** By specifying small chunk size it should be possible to reduce the RAM usage. The following in the YAML file might do it.

>computation:
>  chunking:
>    x: 200
>    y: 200

The above reduced the RAM usage intially, using ~4GB for the first 10min, but it slowly creeped up. The speed of execution dropped significantly. Where previously it analysed 89 tiles in 1.6 min, it was now taking 2 min per tile. At that rate it will not be practical to run it for any significant number of tiles. Increasing the chunk size and the amount of RAM request may find a balance where execution time is acceptable. 

Given that the batch job is runing without the above chunking, it is only an academic issue to find out why interactive job is crashing.

**2.** Another way to circumvent the crash is by using the switch, '--tile-index' to specify the tiles you need to analyse. For example, the whole of Canberra is enclosed in just 2 tiles. These two tiles can be listed in a text file as below.

>15 -40<br>
>15 -41

Then, running the program without the chunking size limitation runs faster. It appears that parallelistaion has been implemented per task which in turn refers to a tile. Hence, if only 2 tiles are given in the 'tile-index' file, then only 2 CPUs are used even if you ask for more.


2. **Existing Output File:** If there are existing files with the same name in the output dir, the program issues a warning but does not overwrite the files (*needs verification*).



## SECTION 3: PBS queue execution on Raijin

## SECTION 4: Interactive API execution on Jupyter notebook

## Import the required modules

In [11]:
%matplotlib inline
import datacube
import matplotlib.pyplot as plt
from skimage import exposure
import numpy as np
from collections import namedtuple

import yaml
from datacube import Datacube
import matplotlib.gridspec as gridspec

import sys
import os.path
from datacube_stats import StatsApp

print(StatsApp.__doc__)



    A StatsApp can produce a set of time based statistical products.
    


In [12]:
import datacube_stats
datacube_stats.__version__

'0.9b1'

### Import a local module 
While the modules loaded above are available to everyone, we need two functions from a module that was in-house developed. In order to use it, you must do the following:
- Git clone the **dea-notebooks** repository to your local workspace.
    - e.g. as /g/data/u46/users/**sa9525**/dea-notebooks, where **sa9525** is to be replaced with your userID.
- Create a soft link from your home dir (e.g. /home/547/sa9525) to the dea-notebooks/Scripts as below.
    - *ln -s /g/data/u46/users/sa9525/dea-notebooks dea-notebooks*


In [13]:
sys.path.append(os.path.expanduser('~/dea-notebooks/Scripts'))
from DEAPlotting import three_band_image

## Configuration File
The entire config for this application resides in a YAML file as given below. More details about its components later...

In [14]:
def main():
    config_yaml = """
    sources:
      - product: ls8_nbar_albers
        measurements: [red, green, blue]
        group_by: solar_day

    date_ranges:
        start_date: 2014-01-01
        end_date: 2014-01-01

    storage:
        # this driver enables in-memory computation
        driver: xarray

        crs: EPSG:3577
        tile_size:
            x: 40000.0
            y: 40000.0
        resolution:
            x: 25
            y: -25
        chunking:
            x: 200
            y: 200
            time: 1
        dimension_order: [time, y, x]

    computation:
        chunking:
            x: 800
            y: 800

#    date_ranges:
#      start_date: 2011-01-01
#      end_date: 2011-01-15

    input_region:
          longitude: [149.05, 149.17]
          latitude: [-35.25, -35.35]

    output_products:
        - name: nbar_mean
          statistic: simple
          statistic_args:
               reduction_function: mean
    """
    return config_yaml
#    print(config_yaml)
    # or manually creating a config dictionary works too

### Create and load the config
Instead of a YAML format as above, the whole data can be specified as a dictionary object (see below). The advantage of YAML is that it is more human readable.

In [15]:
    config_yaml = main()
    config = yaml.load(config_yaml)
#    print(yaml.dump(config, indent=4))
#    print(config)


### Define the datacube stats application

In [16]:
dc = Datacube()
app = StatsApp(config, dc.index);

### Generate the tasks
Do not know yet what the tasks are. Will update it as I learn more!

This example is taking a simple mean of the data spread over the date range. There are other methods that are more complex.

In [17]:
#print('generating tasks')
tasks = app.generate_tasks()
#print (dir(tasks))

### Run the tasks

In [18]:
query = {
        'lat': (-35.25, -35.35),
        'lon': (149.05, 149.17),
        'time':('2017-01-01', '2017-03-15')
        }

In [19]:
def calculate_boundingbox(ds):
    Extent = namedtuple('Extent', ['boundingbox'])
    BoundingBox = namedtuple('BoundingBox', ['left', 'bottom', 'right', 'top'])
    left = ds.x.min().item()
    right = ds.x.max().item()
    top = ds.y.min().item()
    bottom=ds.y.max().item()
    return Extent(boundingbox=BoundingBox(left=left, bottom=bottom, right=right, top=top))

In [20]:
    print('Running tasks. May take some time. Be patient!')
    for task in tasks:
#        print(task)
        # this method is only available for the xarray output driver
        output = app.execute_task(task)
        print("task: ", task)
        ds = output.result['nbar_mean']
#    print(dir(ds))
    ds
    ds.attrs['crs'] = 'EPSG:3577'
    ds.attrs['extent'] = calculate_boundingbox(ds)
    

Running tasks. May take some time. Be patient!


NameError: name 'ds' is not defined

In [ ]:
ds.extent.boundingbox

### Plot it as a grid first

In [ ]:
plt.figure(figsize=(12,8))
gs = gridspec.GridSpec(2,2) # set up a 2 x 2 grid of 4 images for better presentation

ax1=plt.subplot(gs[0,0])
ds.red.isel(time=0).plot(cmap='Reds')

ax2=plt.subplot(gs[1,0])
ds.green.isel(time=0).plot(cmap='Greens')

ax3=plt.subplot(gs[0,1])
ds.blue.isel(time=0).plot(cmap='Blues')

plt.tight_layout()
plt.show()

### Plot it as three bands

In [ ]:
three_band_image(ds, bands = ['red', 'green', 'blue'], time = 0, contrast_enhance=True);